## Decisive Distributions

### Import Dependencies

In [1]:
# Import libraries and dependencies
import os
import pandas as pd
import alpaca_trade_api as tradeapi

# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

%matplotlib inline

In [2]:
# Setup Alpaca API environment
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

In [3]:
type(alpaca_api_key)

str

### Read in stock data using Alpaca API

In [4]:
# Set timeframe to "1Day"
timeframe = "1Day"

# Set start and end datetimes of 1 year, between now and 365 days ago.
start_date = pd.Timestamp("2019-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-05-01", tz="America/New_York").isoformat()

# Set the stock tickers
ticker = ["SPY","LUV","DIS","AAPL","SBUX"]


# Get 1 year's worth of historical data for all stocks
# HINT: Set "limit" to 10000 so all ticker rows are captured from get_bars()
df_ticker = alpaca.get_bars(
    ticker,
    timeframe,
    start_date,
    end_date, 
    limit=10000
).df

# Display sample data
df_ticker.head()

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2019-05-01 04:00:00+00:00,209.88,215.3100,209.230,210.52,64827505,441450,212.670859,AAPL
2019-05-02 04:00:00+00:00,209.84,212.6500,208.130,209.15,32000324,233410,210.155469,AAPL
2019-05-03 04:00:00+00:00,210.89,211.8400,210.230,211.75,21092378,148449,211.161771,AAPL
2019-05-06 04:00:00+00:00,204.29,208.8400,203.500,208.48,32444375,236173,207.309513,AAPL
2019-05-07 04:00:00+00:00,205.88,207.4175,200.825,202.86,38763698,294532,203.568951,AAPL


In [5]:
# Reorganize the DataFrame
# Separate ticker data
SPY = df_ticker[df_ticker['symbol']=='SPY'].drop('symbol', axis=1)
LUV = df_ticker[df_ticker['symbol']=='LUV'].drop('symbol', axis=1)
DIS = df_ticker[df_ticker['symbol']=='DIS'].drop('symbol', axis=1)
AAPL = df_ticker[df_ticker['symbol']=='AAPL'].drop('symbol', axis=1)
SBUX = df_ticker[df_ticker['symbol']=='SBUX'].drop('symbol', axis=1)



# Concatenate the ticker DataFrames
df_ticker = pd.concat([SPY, LUV, DIS, AAPL, SBUX], axis=1, keys=["SPY", "LUV", "DIS", "AAPL", "SBUX"])
# Display sample data
df_ticker.head()

SPY                                       \
                             open    high       low   close     volume   
timestamp                                                                
2019-05-01 04:00:00+00:00  294.72  294.95  291.8000  291.82   71921870   
2019-05-02 04:00:00+00:00  291.68  292.70  289.5200  291.13   65058183   
2019-05-03 04:00:00+00:00  292.82  294.34  291.2998  293.98   56579556   
2019-05-06 04:00:00+00:00  289.25  293.31  288.9000  292.83  107250892   
2019-05-07 04:00:00+00:00  290.15  290.81  285.8100  288.05  145292657   

                                                     LUV                 ...  \
                          trade_count        vwap   open   high     low  ...   
timestamp                                                                ...   
2019-05-01 04:00:00+00:00      337374  293.457138  54.60  54.61  53.855  ...   
2019-05-02 04:00:00+00:00      342564  291.101680  53.99  54.49  53.360  ...   
2019-05-03 04:00:00+00:00      249557  293.438891  53.52  53.93  53.285  ...   
2019-05-06 04:00:00+00:00      390959  291.952803  52.81  53.61  52.720  ...   
2019-05-07 04:00:00+00:00      618594  288.280747  53.04  53.12  51.450  ...   

                               AAPL                           SBUX         \
                             volume trade_count        vwap   open   high   
timestamp                                                                   
2019-05-01 04:00:00+00:00  64827505      441450  212.670859  77.67  78.15   
2019-05-02 04:00:00+00:00  32000324      233410  210.155469  77.66  77.76   
2019-05-03 04:00:00+00:00  21092378      148449  211.161771  77.59  78.40   
2019-05-06 04:00:00+00:00  32444375      236173  207.309513  77.20  78.08   
2019-05-07 04:00:00+00:00  38763698      294532  203.568951  77.57  78.74   

                                                                         
                             low  close   volume trade_count       vwap  
timestamp                                                                
2019-05-01 04:00:00+00:00  77.37  77.52  6678194       50558  77.684302  
2019-05-02 04:00:00+00:00  76.75  77.47  6247550       50730  77.272023  
2019-05-03 04:00:00+00:00  77.34  78.05  4967928       39845  78.009787  
2019-05-06 04:00:00+00:00  77.02  78.06  5607843       46777  77.731052  
2019-05-07 04:00:00+00:00  77.42  77.96  8115808       71206  78.051332  

[5 rows x 35 columns]

### Create a new DataFrame and store the closing prices of each stock.

In [6]:
# Create and empty DataFrame for closing prices
df_closing_prices = pd.DataFrame()

# Fetch the closing prices for all the tickers
for i in ticker:
    df_closing_prices[i] = df_ticker[i]['close']
# Drop the time component of the date
df_closing_prices.index = df_closing_prices.index.date

# Display sample data
df_closing_prices.head()

,SPY,LUV,DIS,AAPL,SBUX
2019-05-01,291.82,53.90,136.38,210.52,77.52
2019-05-02,291.13,53.37,134.14,209.15,77.47
2019-05-03,293.98,53.62,134.33,211.75,78.05
2019-05-06,292.83,53.48,135.00,208.48,78.06
2019-05-07,288.05,52.21,133.44,202.86,77.96


### Calculate the daily returns for each stock using the `pct_change()` function

In [10]:
# Compute daily returns
df_daily_returns = df_closing_prices.pct_change()
df_daily_returns.dropna()

# Display sample data
df_daily_returns.head()

,SPY,LUV,DIS,AAPL,SBUX
2019-05-01,NaN,NaN,NaN,NaN,NaN
2019-05-02,-0.002364,-0.009833,-0.016425,-0.006508,-0.000645
2019-05-03,0.009789,0.004684,0.001416,0.012431,0.007487
2019-05-06,-0.003912,-0.002611,0.004988,-0.015443,0.000128
2019-05-07,-0.016323,-0.023747,-0.011556,-0.026957,-0.001281


### Plot Distributions

In [ ]:
# Visualize the distribution of daily returns across all stocks using a histogram plot
# Hint: To make the plot easier to read, set the alpha arguement to 0.5
# YOUR CODE HERE!

In [ ]:
# Visualize the distribution of daily returns across all stocks using a density plot
# YOUR CODE HERE!